# Create core area masks
This code creates masks for all events in the DeepExtremes database. An event mask includes all pixels that have been part of the respective event at least during one day. The masks are later used to compute the ecosystem response over event-affected area only. This ensures that no other area than the actual event area affect the calculation of the response parameters. This should be more exact compared to the previous bounding-box approach.

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
from xcube.core.store import new_data_store
import os
from datetime import date, timedelta
import numpy.ma as ma
import netCDF4 as nc

We first load the summary table of all DeepExtremes events and create a dataframe of it that fits our purposes. We require the duration of each event to be numeric, we remove events that are shorter than five days as we expect them to have no observable influence on the vegetation parameters we are looking at (because of water and energy stocks in the vegetation) and we convert start and end date to actual datetime objects as we will work with dates.

In [3]:
# Load summary table of DeepExtremes events
df_summary = pd.read_csv("https://s3.bgc-jena.mpg.de:9000/deepextremes/v3/MergedEventStats_landonly.csv")

# Create new column 'duration_days' with numeric values
df_summary['duration_days'] = df_summary['duration'].str.extract(r'(\d+)').astype(int)

# Drop events shorter than 5 days
n_entries = len(df_summary)
df_summary = df_summary[df_summary['duration_days'] > 4]

# Output
print(f"Removed {n_entries - len(df_summary)} entries shorter than 5 days.")
print(f"Final dataframe has {len(df_summary)} entries.")

# Convert date columns from strings to actual datetime objects
df_summary['start_time'] = pd.to_datetime(df_summary['start_time'])
df_summary['end_time'] = pd.to_datetime(df_summary['end_time'])

# Set the label as the index  as unique identifier to iterate over
df_summary.set_index('label', inplace=True)

# Sort dataframe by event label
df_summary = df_summary.sort_index()

Removed 20592 entries shorter than 5 days.
Final dataframe has 17052 entries.


Next, we load the DeepExtremes labelcube that contains all events that were identified in the project. The values of this datacube correspond to the labels of the events from the summary table. Thus, each event is represented as a number of connected pixels that span over the time and space dimensions of the datacube. 

In [4]:
# Load labelcube
ds_blobs = xr.open_zarr('https://s3.bgc-jena.mpg.de:9000/deepextremes/v3/mergedlabels.zarr')

We define a dictionary that contains a list of all time stamps for each event in df_summary. These will be used to retrieve the time slices for each event that will be merged to the core area in the last step.

In [5]:
# Function to create a list of timestamps from given start and end date
def generate_date_list(start_date, end_date):
  """
  Generates a list of daily timestamps (date objects) from a start date to an end date of an event.

  Args:
      start_date (datetime.date): The starting date (inclusive).
      end_date (datetime.date): The ending date (inclusive).

  Returns:
      list: A list of datetime.date objects representing daily timestamps.
  """
  date_list = []
  current_date = start_date
  while current_date <= end_date:
    date_list.append(current_date)
    current_date += timedelta(days=1)
  return date_list

# Initialize an empty dictionary
dict_label_dates = {}

# Iterate over all events in the dataframe df_summary
for index, row in df_summary.iterrows():
    start_date = row['start_time'] # Set start date of respective event
    end_date = row['end_time'] # Set end date of respective event
    dates_list = generate_date_list(start_date, end_date) # Create list of daily time stamps between start and end date
    dict_label_dates[index] = dates_list # Write this list to the dictionary


In [6]:
# Create relative path
current_directory = os.getcwd() # Get the current script's directory

data_directory = os.path.join(current_directory, '..', 'data/masks') # Construct the path to the data directory

The next step is the actual creation of one core area mask for each event in df_summary. As this process is very time and memory consuming, the results will be written to file directly. Each iteration writes a core area mask as a numpy array using the .npy file format. Each of these files will be named after the corresponding event label which will be helpful later on when using these as masks for the calculation of ecosystem response. 

We did not find any other suitable host for the data, so the results will be stored in a separate folder in the data directory of this GitLab repository. The mask-folder is included in the .gitignore file and thus is not part of the version control.

In [8]:
# Loop over each label and its associated event days

for label, event_days in dict_label_dates.items():
    
    # Initialize core_area_mask as a boolean array of the same shape as a single time slice, filled with False
    time_slice_shape = ds_blobs.sel(Ti=event_days[0])['labels'].shape
    core_area_mask = np.zeros(time_slice_shape, dtype=bool)

    # Loop over each date to update the core_area_mask
    for event_day in event_days:
        time_slice = ds_blobs.sel(Ti=event_day)  # Get one time slice of the event per iteration
        event_mask = ma.masked_where(time_slice['labels'] != label, time_slice['labels'])  # Mask time slice where the label exists

        # Update core_area_mask: True where label occurs at least once
        core_area_mask = np.logical_or(core_area_mask, event_mask == label)

    # Convert the final core_area_mask to a general array 
    core_area_mask = np.asarray(core_area_mask)

    # Save the core_area_mask to a file named after the label
    filename = os.path.join(data_directory, f"{label}.npy")
    np.save(filename, core_area_mask)